In [ ]:
import shutil
import os
from zipfile import ZipFile
import pandas as pd


ModuleNotFoundError: No module named 'sklearn'

In [20]:
import pandas as pd

# Load metadata
df = pd.read_csv("Data_Entry_2017.csv")

# Define classification logic
def classify(label):
    if label == "No Finding":
        return 2
    elif "Pneumonia" in label:
        return 0
    else:
        return 1

df["Class"] = df["Finding Labels"].apply(classify)

# Check class distribution
print(df["Class"].value_counts())


Class
2    60361
1    50328
0     1431
Name: count, dtype: int64


In [21]:
# Stratified sampling
df_balanced = (
    df.groupby("Class", group_keys=False)
    .apply(lambda x: x.sample(n=1000, random_state=42))
    .reset_index(drop=True)
)

# Check again
print(df_balanced["Class"].value_counts())


Class
0    1000
1    1000
2    1000
Name: count, dtype: int64


C:\Users\sidch\AppData\Local\Temp\ipykernel_17368\508140848.py:4: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x.sample(n=1000, random_state=42))


In [22]:
df_balanced.to_csv("balanced_dataset.csv", index=False)
print("✅ Saved: balanced_dataset.csv")
df_balanced.head()


✅ Saved: balanced_dataset.csv


,Image Index,Finding Labels,Follow-up #,Patient ID,Patient Age,Patient Gender,View Position,OriginalImage[Width,Height],OriginalImagePixelSpacing[x,y],Unnamed: 11,Class
0,00002360_002.png,Pneumonia,2,2360,33,F,PA,2048,2500,0.171,0.171,NaN,0
1,00022338_002.png,Atelectasis|Pneumonia,2,22338,60,F,PA,2992,2991,0.143,0.143,NaN,0
2,00010530_028.png,Edema|Infiltration|Pneumonia,28,10530,38,F,PA,2658,2725,0.143,0.143,NaN,0
3,00008043_002.png,Atelectasis|Pneumonia,2,8043,59,F,PA,2366,2509,0.143,0.143,NaN,0
4,00026560_010.png,Cardiomegaly|Consolidation|Pneumonia,10,26560,46,F,AP,3056,2544,0.139,0.139,NaN,0


In [28]:
from sklearn.model_selection import train_test_split
import pandas as pd

# Load the balanced dataset
df = pd.read_csv("balanced_dataset.csv")

# First split into train (70%) and temp (30%)
train_df, temp_df = train_test_split(
    df, 
    test_size=0.3, 
    stratify=df["Class"], 
    random_state=42
)

# Then split temp into val (15%) and test (15%)
val_df, test_df = train_test_split(
    temp_df,
    test_size=0.5,
    stratify=temp_df["Class"],
    random_state=42
)

# Save the splits
train_df.to_csv("train.csv", index=False)
val_df.to_csv("val.csv", index=False)
test_df.to_csv("test.csv", index=False)

print("✅ Train:", train_df["Class"].value_counts().to_dict())
print("✅ Val:", val_df["Class"].value_counts().to_dict())
print("✅ Test:", test_df["Class"].value_counts().to_dict())


✅ Train: {1: 700, 2: 700, 0: 700}
✅ Val: {1: 150, 0: 150, 2: 150}
✅ Test: {1: 150, 2: 150, 0: 150}


In [32]:
import os
import pandas as pd
import shutil

# ✅ Step 1: Define your extracted image folders
# Assuming folders are named images_001, images_002, ..., images_012
image_folders = [f"images_{i:03d}" for i in range(1, 13)]

# ✅ Step 2: Recursively index all available image files (even inside subfolders like 'images_001/images/')
file_map = {}
for base_folder in image_folders:
    root = os.path.abspath(base_folder)
    if not os.path.exists(root):
        continue
    for dirpath, _, filenames in os.walk(root):
        for fname in filenames:
            normalized = fname.strip().lower()
            full_path = os.path.join(dirpath, fname)
            file_map[normalized] = full_path

print(f"✅ Indexed {len(file_map)} total image files across all folders.")

# ✅ Step 3: Set class labels
class_map = {
    0: "pneumonia",
    1: "other",
    2: "normal"
}

# ✅ Step 4: Create output folder structure
splits = ["train", "val", "test"]
for split in splits:
    for class_id, class_name in class_map.items():
        os.makedirs(os.path.join("curated_data", split, f"class_{class_id}_{class_name}"), exist_ok=True)

# ✅ Step 5: Copy matching files from image folders to curated_data structure
missing = 0
copied = 0

for split in splits:
    print(f"\n📦 Processing {split}.csv...")
    df = pd.read_csv(f"{split}.csv")

    for _, row in df.iterrows():
        raw_fname = row["Image Index"]
        normalized_fname = raw_fname.strip().lower()
        class_id = row["Class"]
        class_folder = f"class_{class_id}_{class_map[class_id]}"
        dest_path = os.path.join("curated_data", split, class_folder, raw_fname)

        if normalized_fname in file_map:
            src_path = file_map[normalized_fname]
            shutil.copyfile(src_path, dest_path)
            copied += 1
        else:
            print(f"⚠️ Missing: {raw_fname}")
            missing += 1

print(f"\n✅ Done!")
print(f"✅ Total images copied: {copied}")
print(f"⚠️ Total missing files: {missing}")


✅ Indexed 112120 total image files across all folders.

📦 Processing train.csv...

📦 Processing val.csv...

📦 Processing test.csv...

✅ Done!
✅ Total images copied: 3000
⚠️ Total missing files: 0


In [31]:
import os

for folder in ["images_001", "images_002", "images_003"]:
    if os.path.exists(folder):
        print(f"Folder: {folder}")
        print(os.listdir(folder)[:5])  # Print first 5 filenames
        break


Folder: images_001
['images']


In [33]:
import shutil

shutil.make_archive("pneumonia_curated", 'zip', "curated_data")
print("✅ Zipped successfully: pneumonia_curated.zip")


✅ Zipped successfully: pneumonia_curated.zip
